In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import string
import numpy as np

# Download NLTK stopwords and wordnet
nltk.download('stopwords')
nltk.download('wordnet')

# Define the emotions
emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral", "love"]

# Load the dataset
data = pd.read_csv("output.csv")
df = pd.DataFrame(data)
Text = df["Text"]

# Preprocess text data
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['Text'] = df['Text'].apply(preprocess_text)

# Encode the labels
label_encoder = LabelEncoder()
df['emotion'] = label_encoder.fit_transform(df['emotion'])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['emotion'], test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Define the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=maxlen))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(len(emotions), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping and model checkpoint
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
earlyStopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[checkpoint, earlyStopping, reduce_lr])

# Load the best model
model.load_weights('best_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy: {accuracy}')

# Function to classify new text
def classify_text(text):
    text = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, padding='post', maxlen=maxlen)
    prediction = model.predict(padded_sequence)
    emotion = label_encoder.inverse_transform([prediction.argmax()])
    return emotion[0]

# Example usage
new_text = "I love spending time with you!"
predicted_emotion = classify_text(new_text)
print(f"Text: {new_text}")
print(f"Predicted Emotion: {predicted_emotion}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Epoch 1/10
162/162 [==============================] - 119s 686ms/step - loss: 1.5484 - accuracy: 0.4938 - val_loss: 1.4074 - val_accuracy: 0.5301 - lr: 0.0010
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


162/162 [==============================] - 108s 670ms/step - loss: 1.1887 - accuracy: 0.6026 - val_loss: 1.1566 - val_accuracy: 0.6206 - lr: 0.0010
Epoch 3/10
162/162 [==============================] - 109s 671ms/step - loss: 0.8863 - accuracy: 0.7120 - val_loss: 1.1120 - val_accuracy: 0.6414 - lr: 0.0010
Epoch 4/10
162/162 [==============================] - 103s 634ms/step - loss: 0.7083 - accuracy: 0.7723 - val_loss: 1.1238 - val_accuracy: 0.6476 - lr: 0.0010
Epoch 5/10
162/162 [==============================] - 108s 669ms/step - loss: 0.5891 - accuracy: 0.8125 - val_loss: 1.1821 - val_accuracy: 0.6437 - lr: 0.0010
Epoch 6/10
162/162 [==============================] - 109s 672ms/step - loss: 0.5178 - accuracy: 0.8290 - val_loss: 1.1944 - val_accuracy: 0.6414 - lr: 0.0010
Epoch 6: early stopping
Accuracy: 0.6526576280593872
1/1 [==============================] - 3s 3s/step
Text: I love spending time with you!
Predicted Emotion: joy


In [2]:
# Example usage
new_text = "I hate that"
predicted_emotion = classify_text(new_text)
print(f"Text: {new_text}")
print(f"Predicted Emotion: {predicted_emotion}")

1/1 [==============================] - 0s 59ms/step
Text: I hate that
Predicted Emotion: anger


In [3]:
# Example usage
new_text = "he love daisee"
predicted_emotion = classify_text(new_text)
print(f"Text: {new_text}")
print(f"Predicted Emotion: {predicted_emotion}")

1/1 [==============================] - 0s 62ms/step
Text: he love daisee
Predicted Emotion: joy


In [4]:
# Example usage
new_text = "get lost"
predicted_emotion = classify_text(new_text)
print(f"Text: {new_text}")
print(f"Predicted Emotion: {predicted_emotion}")

1/1 [==============================] - 0s 117ms/step
Text: get lost
Predicted Emotion: surprise


In [5]:
# Example usage
new_text = "You'll videotape everything, yeah?"
predicted_emotion = classify_text(new_text)
print(f"Text: {new_text}")
print(f"Predicted Emotion: {predicted_emotion}")

1/1 [==============================] - 0s 62ms/step
Text: You'll videotape everything, yeah?
Predicted Emotion: sadness
